# Cleaning the Data

In [374]:
import pandas as pd
import numpy as np

movies_df = pd.read_csv("../data/output_data/combined_movies.csv")

In [375]:
# removing the first row (empty)
movies_df = movies_df.iloc[1:]

In [376]:
# dropping Poster URL and overview column
movies_df = movies_df.drop(columns=["Poster URL", "Overview"])

In [377]:
# dropping missing values
movies_df = movies_df.dropna()

# dropping zeros (budget and revenue)
movies_df = movies_df[(movies_df["Budget"] != 0) & (movies_df["Revenue"] != 0)]

In [378]:
# converting runtime to numeric
movies_df["Runtime"] = movies_df["Runtime"].str.replace(" min", "").astype(int)

In [379]:
# splitting the released date into year, month, and day
movies_df["Released"] = pd.to_datetime(movies_df["Released"], errors='coerce')
movies_df["Year"] = movies_df["Released"].dt.year
movies_df["Month"] = movies_df["Released"].dt.month
movies_df["Day"] = movies_df["Released"].dt.day

# dropping the original Released column
movies_df = movies_df.drop(columns=["Released"])

In [380]:
# making the genre column one-hot encoded
genre_dummies = movies_df["Genre"].str.get_dummies(sep=', ')
movies_df = pd.concat([movies_df, genre_dummies], axis=1)

# dropping the original genre column
movies_df = movies_df.drop(columns=["Genre"])

In [381]:
# making the MPAA rating one-hot encoded
mpaa_dummies = movies_df["Rated"].str.get_dummies(sep=', ')
movies_df = pd.concat([movies_df, mpaa_dummies], axis=1)

# dropping the original MPAA rating column
movies_df = movies_df.drop(columns=["Rated"])

In [382]:
# creating variable that indicates the movie was made by a big production company
companies = ["Universal Pictures", "Paramount Pictures", "Warner Bros. Pictures", "Walt Disney Pictures", "Walt Disney Feature Animation", "Sony Pictures", "Universal Pictures", "20th Century Fox", "Columbia Pictures", "Lionsgate", "Metro-Goldwyn-Mayer", "Marvel Studios", "Pixar Animation Studios", "DreamWorks Animation", "DreamWorks Pictures", "New Line Cinema", "Focus Features", "Orion Pictures", "United Artists", "A24", "Searchlight Pictures"]
movies_df["Top_Production_Company"] = movies_df["Production Companies"].apply(lambda x: int(any(company in x for company in companies)))

# dropping the original Production Companies column
movies_df = movies_df.drop(columns=["Production Companies"])

In [383]:
# creaing variable that indicates the movie was directed by top 25 director
top_directors = ["Steven Spielberg", "James Cameron", "Russo brohters", "Peter Jackson", "Michael Bay", "David Yates", "Christopher Nolan", "Ridley Scott", "Tim Burton", "J.J. Abrams", "Ron Howard", "Robert Zemeckis", "Jon Favreau", "Sam Raimi", "James Wan", "Chris Columbus", "Roland Emmerich", "Jon Watts", "Gore Verbinski", "Bryan Singer", "Pierre Coffin", "Francis Lawrence", "Clint Eastwood", "George Lucas", "Brad Bird", "M. Night Shyamalan", "Todd Phillips", "Chris Renaud", "Zack Snyder", "Kyle Balda"]
movies_df["Top_Director"] = movies_df["Director"].apply(lambda x: int(any(director in x for director in top_directors)))

# dropping the original Director column
movies_df = movies_df.drop(columns=["Director"])

In [384]:
# creaing variable that indicates the movie was written by top 20 writer

# splitting the column and keeping only the first writer
movies_df["Writer"] = movies_df["Writer"].str.split(',').str[0]


top_writers =  ["Aaron Sorkin", "Quentin Tarantino", "Woody Allen", "Billy Wilder", "Joel Coen", "Ethan Coen", "Oliver Stone", "William Goldman", "Steven Zaillian", "James Cameron", "Mel Brooks", "Paul Schrader", "Ingmar Bergman", "Lawrence Kasdan", "Eric Roth", "Alan Ball", "Paul Thomas Anderson", "Walter Hill", "John Hughes", "David Lynch"]
movies_df["Top_Writer"] = movies_df["Writer"].apply(lambda x: int(any(writer in x for writer in top_writers)))

# dropping the original Director column
movies_df = movies_df.drop(columns=["Writer"])

In [385]:
# converting text variables to strings

movies_df["Title"] = movies_df["Title"].astype("string")
movies_df["Actors"] = movies_df["Actors"].astype("string")
movies_df["Description"] = movies_df["Description"].astype("string")
movies_df["Tagline"] = movies_df["Tagline"].astype("string")

In [386]:
# convering IMDB votes to numeric
movies_df["IMDB Votes"] = movies_df["IMDB Votes"].str.replace(",", "").astype(int)

In [387]:
# converting box office to numeric
movies_df["Box Office"] = movies_df["Box Office"].str.replace("$", "").str.replace(",", "").astype(float)

In [388]:
# splitting the column and keeping only the first language
movies_df["Country"] = movies_df["Country"].str.split(',').str[0]

# creating variable that indicates the movie was made in English
movies_df["Domestic"] = movies_df["Country"].apply(lambda x: int(x == "United States"))
# variable that indicates the movie was made in other language
movies_df["International"] = movies_df["Country"].apply(lambda x: int(x != "United States"))

# dropping the original country column
movies_df = movies_df.drop(columns=["Country"])

In [389]:
# splitting the column and keeping only the first language
movies_df["Language"] = movies_df["Language"].str.split(',').str[0]

# creating variable that indicates the movie was made in English
movies_df["English_Language"] = movies_df["Language"].apply(lambda x: int(x == "English"))
# variable that indicates the movie was made in other language
movies_df["Other_Language"] = movies_df["Language"].apply(lambda x: int(x != "English"))

# dropping the original country column
movies_df = movies_df.drop(columns=["Language"])

In [390]:
import re

# converting the awards column to numeric
movies_df["Oscars_Won"] = 0
movies_df["Oscars_Nominated"] = 0
movies_df["Wins"] = 0
movies_df["Nominations"] = 0

# Extract with regex
for i, row in movies_df.iterrows():
    text = row["Awards"]

    # Oscars
    won_oscar = re.search(r"Won (\d+) Oscar", text)
    nom_oscar = re.search(r"Nominated for (\d+) Oscar", text)
    if won_oscar:
        movies_df.at[i, "Oscars_Won"] = int(won_oscar.group(1))
    if nom_oscar:
        movies_df.at[i, "Oscars_Nominated"] = int(nom_oscar.group(1))

    # General wins
    wins = re.search(r"(\d+) wins?", text)
    if wins:
        movies_df.at[i, "Wins"] = int(wins.group(1))

    # General nominations
    noms = re.search(r"(\d+) nominations?", text)
    if noms:
        movies_df.at[i, "Nominations"] = int(noms.group(1))

# dropping the original Awards column
movies_df = movies_df.drop(columns=["Awards"])

In [391]:
# check data types
print(movies_df.dtypes[:10])
print(movies_df.dtypes[10:20])
print(movies_df.dtypes[20:24])

Title          string[python]
Year                    int32
Runtime                 int64
Actors         string[python]
IMDB Rating           float64
Metascore             float64
IMDB Votes              int64
Box Office            float64
Description    string[python]
Tagline        string[python]
dtype: object
Budget           int64
Revenue          int64
TMDB Rating    float64
Vote Count       int64
Month            int32
Day              int32
Action           int64
Adventure        int64
Animation        int64
Biography        int64
dtype: object
Comedy         int64
Crime          int64
Documentary    int64
Drama          int64
dtype: object


In [392]:
movies = len(movies_df)
print("Number of movies:", movies)

Number of movies: 2816


In [393]:
# check number of columns
columns = len(movies_df.columns)
print("Number of columns:", columns)

Number of columns: 61


In [394]:
movies_df.to_csv("../data/output_data/clean_movies.csv", index=False)